In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
review_file = open('/data/yelp/yelp_academic_dataset_review.json')
reviews = []
for line in review_file:
    reviews.append(json.loads(line))
review_df = pd.DataFrame(reviews)
review_df

In [ ]:
user_file = open('/data/yelp/yelp_academic_dataset_user.json')
users = []
for line in user_file:
    users.append(json.loads(line))
user_df = pd.DataFrame(users)
user_df

In [ ]:
business_file = open('/data/yelp/yelp_academic_dataset_business.json')
businesses = []
for line in business_file:
    businesses.append(json.loads(line))
business_df = pd.DataFrame(businesses)
business_df

In [ ]:
def filte(df: pd.DataFrame, thresh=20):
    users = []
    for k, v in df['user_id'].value_counts().items():
        if v < thresh:
            users.append(k)
    
    items = []
    for k, v in df['business_id'].value_counts().items():
        if v < thresh:
            items.append(k)

    return set(users), set(items)

In [ ]:
drop_users, drop_items = filte(review_df)

review_df.drop(review_df[review_df['user_id'].isin(drop_users)].index, inplace=True)
review_df.drop(review_df[review_df['business_id'].isin(drop_items)].index, inplace=True)

user_df.drop(user_df[user_df['user_id'].isin(drop_users)].index, inplace=True)

business_df.drop(business_df[business_df['business_id'].isin(drop_items)].index, inplace=True)

In [ ]:
review_df['date'] = pd.to_datetime(review_df['date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
review_df = review_df[['stars', 'user_id', 'business_id', 'useful', 'funny', 'cool', 'date']]

In [ ]:
user_df = user_df[['user_id', 'review_count', 'yelping_since', 'useful', 'funny', 'cool']]
# user_df['yelping_since'] = user_df['yelping_since'].apply(date_func)
user_df['yelping_since'] = pd.qcut(pd.to_datetime(user_df['yelping_since'], format='%Y-%m-%d'), 100, labels=False, duplicates='drop')
user_df['review_count'] = pd.qcut(user_df['review_count'].apply(lambda x: int(x)), 100, labels=False, duplicates='drop')
user_df['useful'] = pd.qcut(user_df['useful'].apply(lambda x: int(x)), 100, labels=False, duplicates='drop')
user_df['funny'] = pd.qcut(user_df['funny'].apply(lambda x: int(x)), 100, labels=False, duplicates='drop')
user_df['cool'] = pd.qcut(user_df['cool'].apply(lambda x: int(x)), 100, labels=False, duplicates='drop')

In [ ]:
business_df = business_df[['business_id', 'city', 'state', 'stars', 'review_count', 'is_open', 'categories']]
# business_df['address'] = business_df['address'].append(lambda x: str(x).replace(',', ''))
business_df['stars'] = pd.qcut(business_df['stars'].astype(np.float32), 5, labels=False, duplicates='drop')
business_df['review_count'] = pd.qcut(business_df['review_count'].apply(lambda x: int(x)), 20, labels=False, duplicates='drop')
business_df['categories'] = business_df['categories'].apply(lambda x: '_'.join(str(x).replace(' ', '').split(',')))

In [ ]:
result_df = pd.merge(review_df, user_df, on='user_id')
result_df = pd.merge(result_df, business_df, on='business_id')

In [ ]:
result_df

In [ ]:
result_df.to_csv('/data/yelp/yelp.txt', index=False)